In [409]:
import pandas as pd
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from datasets import DatasetDict, load_from_disk, Dataset, concatenate_datasets
import re

In [410]:
import json
with open('/opt/ml/input/data/KorQuAD_2.1/train/korquad2.1_train_00.json','r') as f:
    train_data = json.load(f)

In [411]:
train_data.keys()

dict_keys(['version', 'data'])

In [412]:
type(train_data)

dict

In [413]:
train_data['version']

'KorQuAD_2.0_train'

In [414]:
len(train_data['data'])

1000

In [415]:
train_data['data'][0].keys()

dict_keys(['title', 'url', 'context', 'raw_html', 'qas'])

In [416]:
train_data['data'][999].keys()

dict_keys(['title', 'url', 'context', 'raw_html', 'qas'])

In [491]:
# http://zeany.net/46
def preprocess_context(example):
    body = re.sub('<title.*?>.*?</title>', '', example, 0, re.I|re.S)
    body = re.sub('<table.*?>.*?</table>', '', body, 0, re.I|re.S)
    body = re.sub('<td.*?>.*?</td>', '', body, 0, re.I|re.S)
    body = re.sub('<li.*?>.*?</li>', '', body, 0, re.I|re.S)
    body = re.sub('<ol.*?>.*?</ol>', '', body, 0, re.I|re.S)
    text = re.sub('<.+?>', '', body, 0, re.I|re.S)
    text = re.sub('\[[0-9]+\]', ' ', text, 0, re.I|re.S)
    text = text.replace("[원본 편집]","")
    text = text.replace("[편집]","")
    text = text.replace("\\'","")
    text = text.replace("펼치기","")
    text = text.replace("버튼 클릭","")
    text = text.split("같이 보기")[0]
    text = text.split("참고 자료")[0]
    text = text.split("외부 링크")[0]
    text = text.split("원본 주소")[0]
    text = text.split("각주")[0]
    text = text.split("참고 문헌")[0]
    text = text.split("함께 보기")[0]
    text = text.split("관련 서적")[0]
    text = text.split("관련 항목")[0]
    space = re.sub('&nbsp;|&lt;|&gt;|\t|\r|\n|\xa0|\u200b', ' ', text)
    one_space = re.sub(' +', ' ', space)
    if one_space[::-1].find('.') == -1:
        return one_space
    if (one_space[::-1].find('.') != 0)|(one_space[::-1].find('.') != ')'):
        last_comma = one_space[:-(one_space[::-1].find('.'))]
        return last_comma
    return one_space

In [492]:
# http://zeany.net/46
def preprocess_answer(example):
    body = re.sub('<title.*?>.*?</title>', '', example, 0, re.I|re.S)
    body = re.sub('<table.*?>.*?</table>', '', body, 0, re.I|re.S)
    body = re.sub('<td.*?>.*?</td>', '', body, 0, re.I|re.S)
    body = re.sub('<li.*?>.*?</li>', '', body, 0, re.I|re.S)
    body = re.sub('<ol.*?>.*?</ol>', '', body, 0, re.I|re.S)
    text = re.sub('<.+?>', '', body, 0, re.I|re.S)
    text = re.sub('\[[0-9]+\]', ' ', text, 0, re.I|re.S)
    text = text.replace("[원본 편집]","")
    text = text.replace("[편집]","")
    text = text.replace("\\'","")
    text = text.replace("펼치기","")
    text = text.replace("버튼 클릭","")
    text = text.split("같이 보기")[0]
    text = text.split("참고 자료")[0]
    text = text.split("외부 링크")[0]
    text = text.split("원본 주소")[0]
    text = text.split("각주")[0]
    text = text.split("참고 문헌")[0]
    text = text.split("함께 보기")[0]
    text = text.split("관련 서적")[0]
    text = text.split("관련 항목")[0]
    space = re.sub('&nbsp;| |\t|\r|\n|\xa0|\u200b', ' ', text)
    one_space = re.sub(' +', ' ', space)
    return one_space

In [493]:
import json
whole_list = []
train_dict = {}
answer_dict = {}
c = 0
for j in range(0,39):
    file_j = ('0'+str(j))[-2:]
    with open('/opt/ml/input/data/KorQuAD_2.1/train/korquad2.1_train_{}.json'.format(file_j),'r') as f:
        train_data = json.load(f)
        for i in range(len(train_data['data'])):
            answers = train_data['data'][i]['qas'][0]
            answer_dict['answer_start'] = [len(preprocess_answer(train_data['data'][i]['context'][:answers['answer']['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_answer(answers['answer']['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='korquad-{}-{}'.format(file_j,count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'])
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if (len(train_dict['context']) <= 2000)&(len(answer_dict['text']) <= 80):
                    if len(train_dict['context']) > answer_dict['answer_start'][0]:
                        if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                            continue
                        else:
                            whole_list.append(train_dict)
                    else:
                        continue
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [494]:
len(whole_list)

12811

In [496]:
with open('/opt/ml/input/data/KorQuAD_2.1/train/preprocessed/train_under2000_full.json','w') as outfile:
    json.dump(whole_list,outfile)

In [497]:
for idx,i in enumerate(range(0,14000,2000)):
    with open('/opt/ml/input/data/KorQuAD_2.1/train/preprocessed/train_under2000_{}.json'.format(idx),'w') as outfile:
        json.dump(whole_list[i:i+2000],outfile)
        print(i,i+2000)

0 2000
2000 4000
4000 6000
6000 8000
8000 10000
10000 12000
12000 14000


In [498]:
import random
whole_list[random.randint(1,len(whole_list))]

{'title': '콜로세움',
 'context': "역사 70년경 베스파시아누스 황제에 의해 건설이 시작되었으며, 80년에 건축이 끝나 100일 축제 기간 동안 그의 아들인 티투스 황제가 개막식을 올렸다. 콜로세움은 높이 42.38m, 둘레 53203CM 등 경기장 내부의 길이 873m와 폭 2355m라는, 당시에 건립된 건축물 가운데 최대의 건축물이었다. 네로 황제의 황금 궁전(도무스 아우레스)의 정원에 있던 인공 호수를 메운 자리에 세워졌다. 본래는 비공식 이름이었던 콜로세움(Colosseum)의 유래에 대해서는 원형 경기장 근처에 있던 네로 황제의 거대한 청동상(Colossus Neronis)과 명칭이 혼동되었다는 설과 '거대하다'는 뜻의 이탈리아어 콜로살레(Colossale)와 어원이 같다는 설이 있는데 중세에는 그 자체가 ‘거대한 건축물’이란 뜻으로 쓰이기도 했다. 이곳에서 열리는 검투사 경기를 보러 찾아드는 5만명의 관객을 수용할 수 있었다. 경기장은 또한 해상 전투를 재현하거나 고전극을 상연하는 무대로도 사용되었다. 검투사들은 보통 노예나 전쟁 포로들 중에서 운동 실력이 출중하고 용맹하게 잘 싸우는 이들로 이루어져 있었는데, 서로 결투를 벌이거나 다양한 종류의 동물을 사냥해 보여 로마 관중들을 즐겁게 해 주었다. 이렇게 살아간다는 것은 위험이 따르는 일이었으나, 검투사가 되면 이득도 있었다. 다른 노예들보다 생활환경이 훨씬 나은 군대식 학교에서 훈련을 받을 수 있었고, 승리를 거둔 검투사들은 영웅 대접을 받음으로써 한편으로는 일체감과 애국심을 불러일으키기도 하고, 다른 한편으로는 공포심을 심어주기 위한 정치적인 목적으로 건립되었다. 콜로세움은 608년까지는 경기장으로 사용되었지만 중세에는 군사적으로 이용되다가 그 이후에는 성당이나 궁전 등의 건축에 사용될 자재의 제공 터가 되었다. 구조 콜로세움의 파노라마. 콜로세움은 4만 명에서 7만 명까지 앉을 수 있으며 4층으로 되어 있다. 1층은 높이 10.5m의 도리아식 반원주, 2층은 높이 11.85

In [502]:
import json
dev_list = []
train_dict = {}
answer_dict = {}
c = 0
for j in range(0,5):
    file_j = ('0'+str(j))[-2:]
    with open('/opt/ml/input/data/KorQuAD_2.1/dev/korquad2.1_dev_{}.json'.format(file_j),'r') as f:
        train_data = json.load(f)
        for i in range(len(train_data['data'])):
            answers = train_data['data'][i]['qas'][0]
            answer_dict['answer_start'] = [len(preprocess_answer(train_data['data'][i]['context'][:answers['answer']['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_answer(answers['answer']['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='korquad-{}-{}'.format(file_j,count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'])
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if (len(train_dict['context']) <= 2000)&(len(answer_dict['text']) <= 80):
                    if len(train_dict['context']) > answer_dict['answer_start'][0]:
                        if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                            continue
                        else:
                            dev_list.append(train_dict)
                    else:
                        continue
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [503]:
len(dev_list)

1501

In [504]:
dev_list[0]

{'title': '공법',
 'context': '구별실익 강력한 중앙집권적 전제국가가 존재하던 로마시대, 근대에는 공사법 구별이 엄격하였다. 반면에 개인주의가 발달한 시대에는 사법의 영역이 확장되었다. 최근 복지국가사상이 발달하면서 공사법 구별이 엄격해지고 있다. 즉, 사적자치가 발달하면 공사법이 부정되고, 국가의 권력이 강력해지면 공사법의 구별이 엄격해진다. 이를 통해서, 공법을 사법에서 굳이 분리하려는 이유는, 국가와 개인이 소송할 때는, 개인과 개인이 소송할 때보다 국가에게 더 유리하게 판결을 하기 위해 공법이 사법에서 분리된다는 것을 알 수 있다. 영미법은 1천년이 넘도록 공사법 구별이 없이 사법으로만 규율하였는데, 20세기에 들어서 공법이 등장하고 있다는 것은, 영미법에서 개인주의나 사적자치가 후퇴하고 국가주의가 강화되고 있다는 의미로도 해석될 수 있다. 공법과 사법의 구별은 구체적 법률관계에 적용할 법규나 법원칙을 결정하기 위해, 또한 분쟁해결을 위한 쟁송수단의 선택과 결정을 위해서 필요하다. 공법과 사법의 관계 최근에 새로이 정립되는 사회법(노동법 및 경제관계법)은 사법과 공법이 융합된 것으로 사법의 공법화 경향을 보여준다. 공법·사법의 이원적 체계를 인정하지 않고 보통법의 지배 원리에 의한 일원적 체계의 영미법계에서는 20세기에 들어오면서 행정법의 성립과 함께 공법과 사법의 구별 문제가 대두되기 시작하였다. 구별 부정설 구별 부정론은 공법 관계도 그 본질에서는 사법 관계와 같은 법률관계인 점에서 차이가 없으며, 노동법·사회법·경제법 등과 같이 공법과 사법의 어느 하나에만 속하지 않는 새로운 법 현상이 출현하는 것을 이유로 공법과 사법의 구별을 부정하는 견해이다. 구별 긍정설 구별 긍정론 가운데 실체법상의 구별 필요성은 우리의 현행 법질서는 공법 관계인가 사법 관계인가에 따라 적용될 법규나 법원칙을 달리하므로 구체적 법률관계에 적용할 법규나 법원칙을 결정하기 위하여 공법과 사법의 구별은 필요하다. 예컨대 행정 주체와 사인(私人)간에 계약을 체결

In [505]:
with open('/opt/ml/input/data/KorQuAD_2.1/dev/preprocessed/valid_under2000_full.json'.format(idx),'w') as outfile:
    json.dump(dev_list,outfile)

In [506]:
for idx,i in enumerate(range(0,1503,501)):
    with open('/opt/ml/input/data/KorQuAD_2.1/dev/preprocessed/valid_under2000_{}.json'.format(idx),'w') as outfile:
        json.dump(dev_list[i:i+501],outfile)
        print(i,i+501)

0 501
501 1002
1002 1503


In [47]:
ds = Dataset.from_json('/opt/ml/input/data/KorQuAD_2.1/train/train.json')

Using custom data configuration default-5f30fd53ca4be2a5
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-5f30fd53ca4be2a5/0.0.0)


In [48]:
dv = Dataset.from_json('/opt/ml/input/data/KorQuAD_2.1/dev/valid.json')

Using custom data configuration default-56fe0ff52f29f51d
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-56fe0ff52f29f51d/0.0.0)


In [98]:
ds['context'][0][ds['answers'][0]['answer_start'][0]]

'나'

In [99]:
ds['answers'][0]['text'][0][0]

'나'

In [100]:
train_dataset = load_from_disk('/opt/ml/input/data/train_dataset')

In [101]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [102]:
concatenate_datasets([train_dataset['train'],ds])

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 42448
})

In [103]:
DatasetDict({'train' : ds, 'validation' : dv})

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 38496
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 4736
    })
})